#Team Members
##1. Name: Malak Sherif Abouelkhair - ID : 20216137
##2. Name: Malak Mahmoud Hassan - ID : 20217010
##3. Name: Ahmed Khaled Moghazy - ID : 20216004
##4. Name: Omar Galal Hassan - ID : 20217006


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_datasets as tfds
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score
import gensim
import re
import string
import nltk
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
!pip install emoji
import emoji
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 11.8 MB/s eta 0:00:00


# Preprocess

##Reading Data

In [ ]:
!wget https://huggingface.co/datasets/stanfordnlp/sentiment140/resolve/refs%2Fconvert%2Fparquet/sentiment140/train/0000.parquet -O train.parquet
train_df = pd.read_parquet('train.parquet')

--2024-12-14 12:56:20--  https://huggingface.co/datasets/stanfordnlp/sentiment140/resolve/refs%2Fconvert%2Fparquet/sentiment140/train/0000.parquet
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.17, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/datasets/sentiment140/c9e118101ee75c018f6acfa538e6b2ff098b0281ed97ce6f57548633b318f4ab?response-content-disposition=inline%3B+filename*%3DUTF-8%27%270000.parquet%3B+filename%3D%220000.parquet%22%3B&Expires=1734440181&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDQ0MDE4MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9kYXRhc2V0cy9zZW50aW1lbnQxNDAvYzllMTE4MTAxZWU3NWMwMThmNmFjZmE1MzhlNmIyZmYwOThiMDI4MWVkOTdjZTZmNTc1NDg2MzNiMzE4ZjRhYj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=B%7E9Sjyzh8cZdoynDOBkxigdhHTRHvypqG54cM6fE9IyFsQkDWILTeeusn

##Checking Neutral records

In [ ]:
!wget https://huggingface.co/datasets/stanfordnlp/sentiment140/resolve/refs%2Fconvert%2Fparquet/sentiment140/test/0000.parquet -O test.parquet
test_df = pd.read_parquet('test.parquet')

--2024-12-14 12:56:24--  https://huggingface.co/datasets/stanfordnlp/sentiment140/resolve/refs%2Fconvert%2Fparquet/sentiment140/test/0000.parquet
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.17, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46124 (45K)
Saving to: ‘test.parquet’

test.parquet        100%[===================>]  45.04K  --.-KB/s    in 0.007s  

2024-12-14 12:56:24 (6.09 MB/s) - ‘test.parquet’ saved [46124/46124]



In [ ]:
train_df['sentiment'].value_counts()

,count
sentiment,
0,800000
4,800000


In [ ]:
test_df['sentiment'].value_counts()

,count
sentiment,
4,182
0,177
2,139


##Elimanating Neutral Records

In [ ]:
test_df = test_df[test_df['sentiment'] != 2]

##Changing Output to 0 for negative tweetsand 1 for positive tweets

In [ ]:
train_df["sentiment"] = train_df["sentiment"].replace({4: 1})
test_df["sentiment"] = test_df["sentiment"].replace({4: 1})

In [ ]:
test_df['sentiment'].value_counts()

,count
sentiment,
1,182
0,177


##Dispalying Data before preprocessing

In [ ]:
train_df

,text,date,user,sentiment,query
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,0,NO_QUERY
1,is upset that he can't update his Facebook by ...,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,0,NO_QUERY
2,@Kenichan I dived many times for the ball. Man...,Mon Apr 06 22:19:53 PDT 2009,mattycus,0,NO_QUERY
3,my whole body feels itchy and like its on fire,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,0,NO_QUERY
4,"@nationwideclass no, it's not behaving at all....",Mon Apr 06 22:19:57 PDT 2009,Karoli,0,NO_QUERY
...,...,...,...,...,...
1599995,Just woke up. Having no school is the best fee...,Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,1,NO_QUERY
1599996,TheWDB.com - Very cool to hear old Walt interv...,Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,1,NO_QUERY
1599997,Are you ready for your MoJo Makeover? Ask me f...,Tue Jun 16 08:40:49 PDT 2009,bpbabe,1,NO_QUERY
1599998,Happy 38th Birthday to my boo of alll time!!! ...,Tue Jun 16 08:40:49 PDT 2009,tinydiamondz,1,NO_QUERY


##Preprocessing Functions

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tweet_tokenizer = TweetTokenizer()

def remove_punc(text):
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

def remove_stopwords(text):
    words = [word for word in word_tokenize(text) if word not in stop_words]
    return " ".join(words)

def lemmatize(text):
    words = [lemmatizer.lemmatize(word) for word in word_tokenize(text)]
    return " ".join(words)

def remove_mentions_rt(text):
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'\bRT\b', '', text)  # Remove retweet indicators
    return text

def normalize_repeated(text):
    return re.sub(r'(.)\1{2,}', r'\1\1', text)  # Replace "goooood" with "good"

def handle_hashtags(text):
    return re.sub(r'#(\w+)', r'\1', text)  # Convert "#Happy" to "Happy"

def remove_emojis(text):
    return emoji.replace_emoji(text, replace="")

def expand_slang(text):
    slang_dict = {"u": "you", "ur": "your", "gr8": "great"}
    return ' '.join([slang_dict.get(word, word) for word in text.split()])

def preprocess(text_col):
    text_col = text_col.apply(remove_punc)
    text_col = text_col.apply(remove_mentions_rt)
    text_col = text_col.apply(normalize_repeated)
    text_col = text_col.apply(handle_hashtags)
    text_col = text_col.apply(remove_emojis)
    text_col = text_col.apply(expand_slang)
    text_col = text_col.apply(lambda x: x.lower())
    text_col = text_col.apply(remove_stopwords)
    text_col = text_col.apply(lemmatize)
    return text_col

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


##Taking half of the dataset as a random sample

In [ ]:
train_df = train_df.sample(frac=0.5, random_state=42)

## Call preprocess function

In [ ]:
train_df["text"] = preprocess(train_df["text"])
test_df["text"] = preprocess(test_df["text"])

# ....................

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#Saving the preprocessed data to drive because it takes time to rerun preprocessing function

In [ ]:
train_df.to_parquet('train_preprocessed.parquet', index=False)
test_df.to_parquet('test_preprocessed.parquet', index=False)

## Uploading Preprocessed data from drive

In [3]:


train_df = pd.read_parquet('/content/drive/MyDrive/DL ASS2/train_preprocessed.parquet')
test_df = pd.read_parquet('/content/drive/MyDrive/DL ASS2/test_preprocessed.parquet')

print(train_df.head())
print(test_df.head())

                                                text  \
0                          chrishasboobs ahh hope ok   
1                 misstoriblack cool tweet apps razr   
2  tiannachaos know family drama lamehey next tim...   
3  school email wont open geography stuff revise ...   
4                               upper airway problem   

                           date             user  sentiment     query  
0  Tue Jun 16 18:18:12 PDT 2009  LaLaLindsey0609          0  NO_QUERY  
1  Mon Apr 06 23:11:14 PDT 2009      sexygrneyes          0  NO_QUERY  
2  Tue Jun 23 13:40:11 PDT 2009       sammydearr          0  NO_QUERY  
3  Mon Jun 01 10:26:07 PDT 2009      Lamb_Leanne          0  NO_QUERY  
4  Sat Jun 20 12:56:51 PDT 2009      yogicerdito          0  NO_QUERY  
                                                text  \
0        stellargirl loovvee dx cool fantastic right   
1                   reading love lee child good read   
2                    ok first assesment fucking rock   
3  kenb

##Dropping useless columns

In [4]:
train_df.drop(columns=["date" , "user" , "query"], inplace = True)
test_df.drop(columns=["date" , "user" , "query"], inplace = True)

##Resetting indexes after columns dropping

In [5]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

##Final data

In [6]:
test_df

,text,sentiment
0,stellargirl loovvee dx cool fantastic right,1
1,reading love lee child good read,1
2,ok first assesment fucking rock,1
3,kenburbary youll love ive mine month never loo...,1
4,mikefish fair enough think perfect,1
...,...,...
354,using latex lot typeset mathematics look hideous,1
355,note hate word hate page hate latex said hate ...,0
356,ahh back real text editing environment latex,1
357,trouble iran see hmm iran iran far away flocko...,0


##Calling Text Vectorizer and applying it to XTrain and XTest which are texts and doing the train test splitting

In [7]:
max_len = 100

vectorizer = TextVectorization(standardize=None, output_sequence_length=max_len)

vectorizer.adapt(train_df["text"].values)

X_train = vectorizer(train_df["text"]).numpy()
X_test = vectorizer(test_df["text"]).numpy()

y_train = train_df["sentiment"].values
y_test = test_df["sentiment"].values

## Getting the Vocabulary from the Vectorizer and checking its size

In [8]:
vocab = vectorizer.get_vocabulary()
print(vocab[:10])

VOCAB_SIZE = len(vocab)
print(VOCAB_SIZE) # unique tokens + 2

['', '[UNK]', 'im', 'day', 'good', 'get', 'like', 'go', 'got', 'dont']
414196


## First Model (Trainable Word Embeddings)

In [9]:
from tensorflow.keras.regularizers import l2

embedding_dim = 50

model1 = Sequential([
    Input(shape=(max_len,)),
    Embedding(input_dim =VOCAB_SIZE, output_dim=embedding_dim, trainable=True ,mask_zero=True),
    LSTM(32 , use_cudnn=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))
])
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 50)             │      20,709,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 32)                  │          10,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,720,457 (79.04 MB)

 Trainable params: 20,720,457 (79.04 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model1.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [11]:
history1 = model1.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))

Epoch 1/2
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 462s 18ms/step - accuracy: 0.7630 - loss: 0.5006 - val_accuracy: 0.8162 - val_loss: 0.3975
Epoch 2/2
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 483s 18ms/step - accuracy: 0.8420 - loss: 0.3746 - val_accuracy: 0.8022 - val_loss: 0.4566


##Model 1 Evalutaion

In [13]:
model1.evaluate(X_test, y_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7886 - loss: 0.4682


[0.45660829544067383, 0.8022283911705017]

##Classification Report for model1

In [14]:
y_pred1 = model1.predict(X_test)
y_pred1 = (y_pred1 > 0.5).astype(int).flatten()
print(classification_report(y_test, y_pred1, target_names=["0", "1"]))

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step
              precision    recall  f1-score   support

           0       0.79      0.81      0.80       177
           1       0.81      0.80      0.80       182

    accuracy                           0.80       359
   macro avg       0.80      0.80      0.80       359
weighted avg       0.80      0.80      0.80       359



## Second Model (With Pre-Trained Word2Vec Embeddings)

# We tried to import GoogleNEws Word2Vec but it had a high precentage of missing Vocab


## Google News

In [ ]:
import kagglehub

path = kagglehub.dataset_download("adarshsng/googlenewsvectors")

print("Path to dataset files:", path)

100%|██████████| 1.64G/1.64G [01:04<00:00, 27.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/adarshsng/googlenewsvectors/versions/1


In [ ]:
from gensim.models import KeyedVectors

# Load the pre-trained Word2Vec embeddings
word2vec_path = f"{path}/GoogleNews-vectors-negative300.bin"
word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

embedding_dim = word2vec.vector_size  # The size of each embedding vector
print(f"Embedding Dimension: {embedding_dim}")

Embedding Dimension: 300


In [ ]:
misses = 0

embedding_matrix = np.zeros((VOCAB_SIZE, embedding_dim))
for i in range(VOCAB_SIZE):
    word = vocab[i]
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]
    else:
        misses += 1

print("Misses =", misses)


Misses = 362499


In [ ]:
print(f"percentage of missing words: {(misses / VOCAB_SIZE):.2f}","%")

percentage of missing words: 0.88 %


Google News embeddings are unsuitable for tweets due to their focus on formal text, lacking the slang, abbreviations, and sentiment patterns essential for analyzing informal social media content

# We found a Word2vec we could use on Github and we imported it and used it

In [ ]:
import gdown


file_id = '1lw5Hr6Xw0G0bMT1ZllrtMqEgCTrM7dzc'

url = f'https://drive.google.com/uc?id={file_id}'

output = 'word2vec_file.bin'

gdown.download(url, output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1lw5Hr6Xw0G0bMT1ZllrtMqEgCTrM7dzc
From (redirected): https://drive.google.com/uc?id=1lw5Hr6Xw0G0bMT1ZllrtMqEgCTrM7dzc&confirm=t&uuid=13198539-1cf8-49a2-b9b7-e772be40767b
To: /content/word2vec_file.bin
 57%|█████▋    | 2.80G/4.90G [00:24<00:08, 241MB/s]

KeyboardInterrupt: 

# We saved it to drive becuase it was 5Gb and then we uploaded it from Drive

In [12]:
from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/DL ASS2/word2vec_file.bin', binary=True, unicode_errors='ignore')

embedding_dim = word2vec.vector_size  # The size of each embedding vector
print(f"Embedding Dimension: {embedding_dim}")

Embedding Dimension: 400


# The percentage of missing words is much less than Google News and this was the best we could do

In [ ]:

misses = 0

embedding_matrix = np.zeros((VOCAB_SIZE, embedding_dim))
for i in range(VOCAB_SIZE):
    word = vocab[i]
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]
    else:
        misses += 1

print("Misses =", misses)


Misses = 298918


In [ ]:
print(f"percentage of missing words: {(misses / VOCAB_SIZE):.2f}","%")

percentage of missing words: 0.72 %


#model2 using the Word2vec

In [ ]:
model2 = Sequential([
    Input(shape=(max_len,)),
    Embedding(input_dim=VOCAB_SIZE, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False, mask_zero=True),
    LSTM(32, use_cudnn=False),
    Dense(1, activation='sigmoid')
])

model2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 100, 400)            │     165,678,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          55,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 165,733,857 (632.22 MB)

 Trainable params: 55,457 (216.63 KB)

 Non-trainable params: 165,678,400 (632.01 MB)

In [ ]:
model2.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
history2 = model2.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))

Epoch 1/2
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 344s 14ms/step - accuracy: 0.7776 - loss: 0.4640 - val_accuracy: 0.8189 - val_loss: 0.3934
Epoch 2/2
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 335s 13ms/step - accuracy: 0.8032 - loss: 0.4228 - val_accuracy: 0.8329 - val_loss: 0.3721


#Evaluating model2 , it shows better results and no overfitting

In [ ]:
model2.evaluate(X_test, y_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8284 - loss: 0.3813


[0.3720570504665375, 0.8328690528869629]

#Saving the models weights to drive to use in the LSTM Forward functions

In [ ]:
model2.save_weights('model2.weights.h5')

In [ ]:
y_pred = model2.predict(X_test)
y_pred = (y_pred > 0.5).astype(int).flatten()
print(classification_report(y_test, y_pred, target_names=["0", "1"]))

#Building a Sequential layers to give the weights of model2 to so that we dont have to run model2 everytime

In [15]:
# Step 1: Recreate the model architecture
model22 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_len,)),
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=embedding_dim, mask_zero=True, trainable=False),
    tf.keras.layers.LSTM(32, use_cudnn=False),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



# Step 2: Load the weights into the model
model22.load_weights('/content/drive/MyDrive/DL ASS2/model2.weights.h5')
print("Weights successfully loaded into model2.")

Weights successfully loaded into model2.


In [18]:
model22.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy'])

#Same Evaluation and classification report as model2

In [19]:
model22.evaluate(X_test, y_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.8284 - loss: 0.3813


[0.3720570504665375, 0.8328690528869629]

In [16]:
y_pred = model22.predict(X_test)
y_pred = (y_pred > 0.5).astype(int).flatten()
print(classification_report(y_test, y_pred, target_names=["0", "1"]))

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step
              precision    recall  f1-score   support

           0       0.83      0.84      0.83       177
           1       0.84      0.83      0.83       182

    accuracy                           0.83       359
   macro avg       0.83      0.83      0.83       359
weighted avg       0.83      0.83      0.83       359



#LSTM Forward Function and predict Function

In [28]:

# Extract weights from the trained model
# Assuming `model2` is already trained

# Extract layers
embedding_layer = model22.layers[0]
lstm_layer = model22.layers[1]
dense_layer = model22.layers[2]  # Last layer

# Extract LSTM weights
W_xh, W_hh, b_h = lstm_layer.get_weights()
units = W_hh.shape[0]
W_xi, W_xf, W_xc, W_xo = np.split(W_xh, 4, axis=1)
W_hi, W_hf, W_hc, W_ho = np.split(W_hh, 4, axis=1)
b_i, b_f, b_c, b_o = np.split(b_h, 4)

# Extract Dense layer weights
dense_weights = dense_layer.get_weights()
W_dense, b_dense = dense_weights[0], dense_weights[1]  # Weights and bias

# LSTM forward function
def LSTM_forward(x, mask):
    h_t = tf.zeros((units,), dtype=tf.float32)  # Hidden state
    c_t = tf.zeros((units,), dtype=tf.float32)  # Cell state

    for t in range(x.shape[0]):
        if mask[t] == 0:  # Skip masked timesteps
            continue

        x_t = tf.reshape(x[t], (1, -1))  # Ensure correct shape for multiplication

        i_t = tf.sigmoid(tf.matmul(x_t, W_xi) + tf.matmul(tf.reshape(h_t, (1, -1)), W_hi) + b_i)
        f_t = tf.sigmoid(tf.matmul(x_t, W_xf) + tf.matmul(tf.reshape(h_t, (1, -1)), W_hf) + b_f)
        c_hat_t = tf.tanh(tf.matmul(x_t, W_xc) + tf.matmul(tf.reshape(h_t, (1, -1)), W_hc) + b_c)
        o_t = tf.sigmoid(tf.matmul(x_t, W_xo) + tf.matmul(tf.reshape(h_t, (1, -1)), W_ho) + b_o)

        c_t = f_t * c_t + i_t * c_hat_t
        h_t = o_t * tf.tanh(c_t)

    h_t = tf.reshape(h_t, (1, -1))
    output = tf.sigmoid(tf.matmul(h_t, W_dense) + b_dense)
    return output.numpy()

# Predict function
def predict(batch):
    outputs = []
    for i in range(batch.shape[0]):
        x = batch[i]
        mask = (x.sum(axis=1) != 0).astype(int)
        output = LSTM_forward(x, mask)
        outputs.append(output)
    return np.array(outputs)





#Calling LSTM Forward Function and predict function and showing the output

In [34]:
# Test predictions
X_test_embedded = embedding_layer(X_test).numpy()
test_data_sample = X_test_embedded[:10]

custom_predictions = predict(test_data_sample)
model_predictions = model22.predict(X_test[:10])



for i in range(len(custom_predictions)):
    print(f"Custom Prediction: {custom_predictions[i]}, Model Prediction: {model_predictions[i]}")
    print(f"Custom Prediction: {(custom_predictions[i] >= 0.5).astype(int) }, Model Prediction: {(model_predictions[i] >= 0.5).astype(int)}")
    print(f"Custom Prediction: {(custom_predictions[i] >= 0.5).astype(int) == (model_predictions[i] >= 0.5).astype(int)}")
    print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Custom Prediction: [[0.9822702]], Model Prediction: [0.9790947]
Custom Prediction: [[1]], Model Prediction: [1]
Custom Prediction: [[ True]]

Custom Prediction: [[0.9449061]], Model Prediction: [0.9449061]
Custom Prediction: [[1]], Model Prediction: [1]
Custom Prediction: [[ True]]

Custom Prediction: [[0.71786207]], Model Prediction: [0.71786207]
Custom Prediction: [[1]], Model Prediction: [1]
Custom Prediction: [[ True]]

Custom Prediction: [[0.23820212]], Model Prediction: [0.2979875]
Custom Prediction: [[0]], Model Prediction: [0]
Custom Prediction: [[ True]]

Custom Prediction: [[0.4124951]], Model Prediction: [0.5372213]
Custom Prediction: [[0]], Model Prediction: [1]
Custom Prediction: [[False]]

Custom Prediction: [[0.8859251]], Model Prediction: [0.8936246]
Custom Prediction: [[1]], Model Prediction: [1]
Custom Prediction: [[ True]]

Custom Prediction: [[0.06008811]], Model Prediction: [0.06008812]
Custom Prediction: [[0]], Model Predictio

for the first 10 samples we got a 0.1 diffrence which was told to be acceptable..

In [35]:
# Check the mean absolute difference
difference = np.abs( (custom_predictions >= 0.5).astype(int).flatten() - (model_predictions >= 0.5).astype(int).flatten()).mean()
print("Mean Absolute Difference:", difference)

Mean Absolute Difference: 0.1


In [23]:
# Flatten y_test[:10] to ensure it's a 1D array
y_true = y_test[:10].flatten()

# Convert predictions to binary (0 or 1) and ensure they are also 1D
model_binary_predictions = (model_predictions >= 0.5).astype(int).flatten()
custom_binary_predictions = (custom_predictions >= 0.5).astype(int).flatten()

# Generate classification reports
print("Model Predictions Classification Report:")
print(classification_report(y_true, model_binary_predictions))

print("\nCustom Predictions Classification Report:")
print(classification_report(y_true, custom_binary_predictions))


Model Predictions Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.89      0.94         9

    accuracy                           0.90        10
   macro avg       0.75      0.94      0.80        10
weighted avg       0.95      0.90      0.91        10


Custom Predictions Classification Report:
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       1.00      0.78      0.88         9

    accuracy                           0.80        10
   macro avg       0.67      0.89      0.69        10
weighted avg       0.93      0.80      0.84        10



#Time Comparision between model predictions and custom predictions

In [24]:
import time

# Measure time for custom LSTM_forward predictions
start_time = time.time()
custom_predictions = predict(test_data_sample)
custom_time = time.time() - start_time

# Measure time for model predictions
start_time = time.time()
model_predictions = model22.predict(X_test[:10])
model_time = time.time() - start_time

print(f"Custom LSTM_forward Time: {custom_time:.4f} seconds")
print(f"Model Prediction Time: {model_time:.4f} seconds")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Custom LSTM_forward Time: 0.3824 seconds
Model Prediction Time: 0.0643 seconds


In [26]:
pip install memory_profiler

#Memory Comparision between model predictions and custom predictions

In [27]:
from memory_profiler import memory_usage

# Measure memory usage for custom predictions
def run_custom_predictions():
    return predict(test_data_sample)

custom_memory = memory_usage(run_custom_predictions)

# Measure memory usage for model predictions
def run_model_predictions():
    return model22.predict(X_test[:10])

model_memory = memory_usage(run_model_predictions)

print(f"Custom LSTM_forward Memory Usage: {max(custom_memory) - min(custom_memory):.4f} MiB")
print(f"Model Prediction Memory Usage: {max(model_memory) - min(model_memory):.4f} MiB")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Custom LSTM_forward Memory Usage: 0.5977 MiB
Model Prediction Memory Usage: 0.3555 MiB


# -Time:

##Custom LSTM_forward was significantly slower due to Python loops and manual computation for each timestep.

##Built-in model.predict leveraged TensorFlow’s optimized computation graph, leading to faster inference.

# - Memory:

##Custom implementation used slightly more memory due to temporary variables for intermediate computations.

##The built-in method was more efficient and streamlined.




# - Conclusion: While the custom implementation provides insights into the LSTM mechanics, the built-in methods are far more efficient for practical applications.

In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/DL ASS2"))

['test_preprocessed.parquet', 'train_preprocessed.parquet', 'word2vec_file.bin', 'model2.weights.h5']
